In [1]:
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os
import archive.candlestick_patterns_v2 as cp

In [6]:

# Load environment variables from .env file
load_dotenv()
# Access the database URL
db_url = os.getenv('DB_URL')

# Create connect sql database
def create_connection_database(db_url):
    # Database connection string
    db_url = db_url
    # Create an SQLAlchemy engine
    return create_engine(db_url)

# read data from the PostgreSQL table
try:
   with create_connection_database(db_url).connect() as connection:
    table_name = "nifty_top_500_stocks"
    query = f"SELECT * from {table_name};"
    df = pd.read_sql_query(query, con=create_connection_database(db_url))
    print("data pulled from sql table")
except Exception as e:
    print(f"Error: {e}")

data pulled from sql table


In [7]:
def sma(dataframe,column_name,timeperiod):
    dataframe[f"SMA_{timeperiod}"] = dataframe[column_name].rolling(window=timeperiod).mean()
    return dataframe

def ema(dataframe,column_name,timeperiod):
    dataframe[f"EMA_{timeperiod}"] = dataframe[column_name].ewm(span=timeperiod, adjust=False).mean()
    return dataframe

In [8]:
#Moving average crossover system
# The entry and exit rules for the crossover system is as stated below:
# Rule 1) – Buy (fresh long) when the short term moving averages turns greater than the long term moving average. Stay in the trade as long as this condition is satisfied
# Rule 2) – Exit the long position (square off) when the short term moving average turns lesser than the longer-term moving average

#25 day EMA with 50 days EMA – use this to identify medium-term trade (upto few weeks)

def moving_average_crossover(dataframe,column_name,short_time,long_time):

    ema(dataframe,column_name,timeperiod=short_time)
    ema(dataframe,column_name,timeperiod=long_time)
    dataframe["ma_crossover"] = dataframe[f'EMA_{short_time}']-dataframe[f'EMA_{long_time}']
    return dataframe

# Apply the function at group level
df = df.sort_values(by=['SYMBOL','DATE'])
df2 = df.groupby('SYMBOL').apply(lambda x: moving_average_crossover(x, 'CLOSE', 25, 50))

display(df2)

C:\Users\megha\AppData\Local\Temp\ipykernel_21584\602658626.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = df.groupby('SYMBOL').apply(lambda x: moving_average_crossover(x, 'CLOSE', 25, 50))


DATE SERIES   OPEN    HIGH     LOW  PREV. CLOSE  \
SYMBOL                                                                   
360ONE    247    2023-12-22     EQ  661.0  664.85  647.05       635.70   
          246    2023-12-26     EQ  659.0  660.85  648.00       657.90   
          245    2023-12-27     EQ  657.9  678.50  647.65       651.50   
          244    2023-12-28     EQ  676.6  715.60  671.45       674.45   
          243    2023-12-29     EQ  708.0  715.00  687.10       708.80   
...                     ...    ...    ...     ...     ...          ...   
ZYDUSLIFE 128393 2024-12-27     EQ  958.0  985.50  958.00       957.65   
          130774 2024-12-30     EQ  973.2  979.60  952.50       972.40   
          131911 2024-12-31     EQ  960.1  976.75  960.10       956.95   
          133057 2025-01-01     EQ  971.0  980.00  965.30       971.70   
          134209 2025-01-02     EQ  978.4  992.35  966.85       973.80   

                     LTP   CLOSE    VWAP   52W H   52W L   VOLUME  \
SYMBOL                                                              
360ONE    247     655.90  657.90  654.22  1960.0  395.10   611023   
          246     651.80  651.50  653.57  1960.0  395.10   151516   
          245     676.00  674.45  670.11  1960.0  395.10   614017   
          244     709.60  708.80  693.62  1960.0  395.10   638810   
          243     710.00  709.85  706.13  1960.0  395.10   455697   
...                  ...     ...     ...     ...     ...      ...   
ZYDUSLIFE 128393  973.15  972.40  975.50  1324.3  666.75  1074014   
          130774  963.50  956.95  958.48  1324.3  685.00  4345745   
          131911  971.00  971.70  970.91  1324.3  685.00   960218   
          133057  972.20  973.80  973.23  1324.3  685.00   436960   
          134209  990.65  990.75  981.37  1324.3  685.00  1012522   

                         VALUE  NO OF TRADES     SYMBOL      EMA_25  \
SYMBOL                                                                
360ONE    247     3.997459e+08         25691     360ONE  657.900000   
          246     9.902610e+07         12622     360ONE  657.407692   
          245     4.114562e+08         35319     360ONE  658.718639   
          244     4.430910e+08         27630     360ONE  662.571051   
          243     3.217830e+08         20339     360ONE  666.207894   
...                        ...           ...        ...         ...   
ZYDUSLIFE 128393  1.047700e+09         39667  ZYDUSLIFE  975.243838   
          130774  4.165318e+09         85162  ZYDUSLIFE  973.836620   
          131911  9.322820e+08         39337  ZYDUSLIFE  973.672265   
          133057  4.252622e+08         22934  ZYDUSLIFE  973.682090   
          134209  9.936580e+08         29835  ZYDUSLIFE  974.995007   

                      EMA_50  ma_crossover  
SYMBOL                                      
360ONE    247     657.900000      0.000000  
          246     657.649020     -0.241327  
          245     658.307882      0.410757  
          244     660.287965      2.283087  
          243     662.231574      3.976320  
...                      ...           ...  
ZYDUSLIFE 128393  991.594353    -16.350514  
          130774  990.235751    -16.399131  
          131911  989.508858    -15.836594  
          133057  988.892825    -15.210734  
          134209  988.965655    -13.970649  

[134212 rows x 18 columns]

In [ ]:
df.SYMBOL.unique()
df_symbol = df[df.SYMBOL == "360ONE"].sort_values(by="DATE")
df_symbol.tail()

In [32]:
# Volume confirmation
def volume_confirmation(days,df):
    """Check if current volume exceeds 10-day average volume."""
    df['Avg_Volume'] = df['VOLUME'].rolling(window=days).mean()
    df['Volume_Confirmed'] = df['VOLUME'] > 1.1 * df['Avg_Volume']
    return df 


def process_dataframe(df):
    patterns = {
        "Doji": [],
        "Four Price Doji": [],
        "Long Legged Doji": [],
        "Gravestone Doji": [],
        "Dragonfly Doji": [],
        "Hammer": [],
        "Hanging Man": [],
        "Bullish Marubozu": [],
        "Bearish Marubozu": [],
    }

    for _, row in df.iterrows():
        candle = cp.SingleCandlePattern(row)
        patterns["Doji"].append(candle.is_doji())
        patterns["Four Price Doji"].append(candle.is_four_price_doji())
        patterns["Long Legged Doji"].append(candle.is_long_legged_doji())
        patterns["Gravestone Doji"].append(candle.is_gravestone_doji())
        patterns["Dragonfly Doji"].append(candle.is_dragonfly_doji())
        patterns["Hammer"].append(candle.is_hammer())
        patterns["Hanging Man"].append(candle.is_hanging_man())
        patterns["Bullish Marubozu"].append(candle.is_bullish_murubozu())
        patterns["Bearish Marubozu"].append(candle.is_bearish_murubozu())

    # Add results to the DataFrame
    for pattern, results in patterns.items():
        df[pattern] = results
    
    single_candle_patterns_columns = ["Doji",
        "Four Price Doji",
        "Long Legged Doji",
        "Gravestone Doji",
        "Dragonfly Doji",
        "Hammer",
        "Hanging Man",
        "Bullish Marubozu",
        "Bearish Marubozu"]
    df = df.sort_values(by=['SYMBOL','DATE'])
    df['Single_Candlestick_pattern'] = df[single_candle_patterns_columns].any(axis=1)
    df = volume_confirmation(10,df)
    return df

In [ ]:
processed_df = process_dataframe(df)
display(processed_df.head())

In [ ]:
df_date = df[df['DATE']==df.DATE.min()]
df_date

In [ ]:
df_date_1 = df_date[(df_date['Single_Candlestick_pattern']==True) & (df_date['Volume_Confirmed']==True) ]
df_date_1